## DATA

In [9]:
with open("data/all_book_data.txt") as f:
    book_data = f.read()

In [36]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=5000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

In [37]:
# text_splitter.split_text(book_data)
splitted_books_data = text_splitter.split_text(book_data)

In [43]:
import nltk
num_words = [len(nltk.word_tokenize(txt)) for txt in splitted_books_data]

In [45]:
max(num_words),min(num_words)

(1158, 435)

In [55]:
def chunk_transcripts(json_data, chunk_size):
    all_chunks = {}

    for key, data in json_data.items():
        transcript_chunks = []
        current_chunk_text = ""
        current_chunk_start_time = None
        word_count = 0

        for item in data:
            text = item['text']
            start_time = item['start']

            words = text.split()

            if word_count + len(words) <= chunk_size:
                current_chunk_text += " " + text
                word_count += len(words)
                if current_chunk_start_time is None:
                    current_chunk_start_time = start_time
            else:
                transcript_chunks.append({'text': current_chunk_text.strip(), 'start_time': current_chunk_start_time})
                current_chunk_text = text
                current_chunk_start_time = start_time
                word_count = len(words)

        if current_chunk_text:
            transcript_chunks.append({'text': current_chunk_text, 'start_time': current_chunk_start_time})

        all_chunks[key] = transcript_chunks

    return all_chunks

In [56]:
import json

with open('data/misc_transcripts.json', 'r') as f:
    data = json.load(f)

In [68]:
import glob
import json
directory_path = 'data'  # Update this to your directory
json_pattern = '*.json'
json_files = glob.glob(f"data/chunked_*.json")

transcripts_text = []

for jf in json_files:
    with open(f'{jf}', 'r') as f:
        data = json.load(f)
    for youtube_id,texts in data.items():
        for text in texts:
            curr_text = text['text']
            if len(curr_text.split()) <=100: continue
            transcripts_text.append(curr_text)

In [3]:
import wandb
from dotenv import load_dotenv,find_dotenv
import os

load_dotenv(find_dotenv(),override=True)
run = wandb.init()
artifact = run.use_artifact('WGQA_T5/model-registry/Modified HyDE model collection:v0', type='model')
artifact_dir = artifact.download()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/athekunal/.netrc


wandb: Downloading large artifact Modified HyDE model collection:v0, 937.51MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:17.4


In [6]:
from transformers import AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

pipe = pipeline("text-generation", model="artifacts/checkpoint-xm04zxnt:v8",tokenizer=tokenizer)

In [7]:
def get_mod_HyDE_answer(question):
    out = pipe(question,max_new_tokens=300,do_sample=True,min_new_tokens=10)
    return out[0]['generated_text']

In [8]:
get_mod_HyDE_answer("What is valuation? ")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"What is valuation? \xa0 \xa0 \xa0 \xa0\xa0 What you need to know are three dimensions, each is the most obvious you can find. \xa0 \xa0\xa0 Do we need a value per se that affects other people? Are there two dimensions, one is that people value each other, and the other is that they value each other's life cycle, because that can change over time.If the answer on a life cycle cannot be answered, would this be the question that comes up again when analyzing the world? Would the answer be the one that determines what is a life cycle? Are there multiple dimensions when estimating a life cycle? The first can be described as looking at people with different lives and not the same person. Two of the most important dimensions are your life cycle, where you decide what is the one you value right versus the one that you think changes your life cycle and how it affects your business prospects. The second dimension is your life cycle, where you decide how the world will change if you get bored wi

In [9]:
get_mod_HyDE_answer("What is valuation ")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"What is valuation ive no way to answer it and no way of giving you feedback about pricing so you come up with a way to estimate the value of an asset and estimate what is the price to book it's basically going to be that you will get a valuation for something right so with my approach I start with price to book and I'm going to say I'll value an asset not only because of its historical value but because an asset is priced based upon data so I'm going to let you be clear what you're trying to do but you're going to be looking at different ways of estimating prices and I'm going to let you be clear on your value but I will start with a pricing model because the best way to come up with pricing models is to say to value an asset or you know where each of these models comes from what you might call valuation models that look like you think that's boring but it might be a good way to value an asset but that was all I wanted you not to value an asset and think of pricing model you don't val